## Transfer learning using VGG16 with trained weights on imagenet dataset
#### Steps :
1.  Removing fully connected layers for classification on top
2.   Freezing the weights of all convulation blocks by making them untrainable
3.   Adding fully connected layer on top of freezed convulation blocks and training them to fit our dataset
4.   Unfreezing the top convulation block and fine-tuning it using our dataset : not needed, already getting good results by just training the fully connected layers on top
    *   Fine-tuning only top convulation layers because lower layers learn general features (like edge, curve, etc.) while top layers learn features which are more dataset specific
 


In [ ]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
def plot_loss_accuracy(history):
    '''
    A function to plot train and validation loss against epochs of training
    '''
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,max(plt.ylim())+0.5])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
# using ImageDataGenerator to load data from the disk


train_val_directory_path = 'chart_images_dataset/charts/train_val'

train_csv_path = 'train.csv'
val_csv_path   = 'val.csv'

train_df = pd.read_csv(train_csv_path)
val_df   = pd.read_csv(val_csv_path)

IMAGE_SIZE = 224 # VGG input size
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) # 3 channels
NUM_OF_CLASSES = 5


# preprocessing the generated image batches with VGG16 preprocessing function 
# VGG16 preprocessing function : converts from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling. 
image_generator = ImageDataGenerator( preprocessing_function=tf.keras.applications.vgg16.preprocess_input  )

train_data_gen = image_generator.flow_from_dataframe(
    train_df, 
    directory=train_val_directory_path, 
    x_col='image_filename', 
    y_col='type', 
    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
    color_mode='rgb', 
    class_mode='categorical', 
    batch_size=32, 
    shuffle=False, # already shuffled 
    seed=None, 
    validate_filenames=True
)

val_data_gen = image_generator.flow_from_dataframe(
    val_df, 
    directory=train_val_directory_path, 
    x_col='image_filename', 
    y_col='type', 
    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
    color_mode='rgb', 
    class_mode='categorical', 
    batch_size=32, 
    shuffle=False, # already shuffled 
    seed=None,
    validate_filenames=True

)

In [ ]:
# checking/ validating some images generated by datagenerator

print ('indices of categories : {}'.format(train_data_gen.class_indices))

# This function will plot 4 images along with their labels.
def plotImages(image_datas,y_list):
    f, axarr = plt.subplots(2,2)
    axarr[0,0].imshow(image_datas[0], cmap='gray')
    axarr[0,0].set_title(y_list[0])
    axarr[0,0].axis('off')
    axarr[0,1].imshow(image_datas[1], cmap='gray')
    axarr[0,1].set_title(y_list[1])
    axarr[0,1].axis('off')
    axarr[1,0].imshow(image_datas[2], cmap='gray')
    axarr[1,0].set_title(y_list[2])
    axarr[1,0].axis('off')
    axarr[1,1].imshow(image_datas[3], cmap='gray')
    axarr[1,1].set_title(y_list[3])
    axarr[1,1].axis('off')

sample_training_images, y_list = next(train_data_gen)
sample_training_images = np.squeeze(sample_training_images) # to reduce dimension
print("shape of a batch given by image data generator : {}".format(sample_training_images.shape))

plotImages(sample_training_images[:4],y_list[:4])

In [ ]:
# loading VGG16 with weights trained on imagenet dataset and w/o top classification layers 
base_VGG16 = tf.keras.applications.VGG16(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_VGG16.summary()

Training only top dense layers for predicting class

In [ ]:
def create_VGG_based_model():
    
    base_VGG16.trainable = False

    model = tf.keras.models.Sequential([
        base_VGG16,
        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_OF_CLASSES, activation='softmax') 
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    return model

In [ ]:
VGG_based_model = create_VGG_based_model()
VGG_based_model.summary()

In [ ]:
# training

VGG_based_model_checkpoint_filepath = 'VGG_based_model/checkpoint/'
VGG_based_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=VGG_based_model_checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              verbose=0, mode='auto')

VGG_based_model_history = VGG_based_model.fit(train_data_gen, epochs=50, 
                    validation_data=val_data_gen,
                    callbacks=[VGG_based_model_checkpoint_callback, early_stopping_callback])